# Fashion MNIST Model Training & Prediction

This notebook trains a neural network on the Fashion MNIST dataset and uses it to predict clothing types from images.

In [1]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps
import h5py
import zipfile

print("cwd:", os.getcwd())
print("files (top-level):", os.listdir('.'))

# Load Fashion MNIST dataset
print("Loading Fashion MNIST dataset...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize and reshape data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile and train
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("\nTraining model...")
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1, verbose=1)

# Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc*100:.2f}%")

# Save the model
model_path = 'mi_modelo.keras'
model.save(model_path)
print(f"Model saved as: {model_path}")

cwd: c:\Users\Miguel\Desktop\Miguelon\GH\Machine_Learning_Projects
files (top-level): ['.git', '.venv', 'Artificial_Brain_Fashion.ipynb', 'Camino_Del_Guerrero_No_Numpy.ipynb', 'Camino_Del_Guerrero_Numpy.ipynb', 'Conv_MaxPooling2D.ipynb', 'First_Neural_Network_OR_Gate.py', 'Hello.py', 'Neural_Network_AND_Gate.py', 'TrampaMortalFashionMNIST.ipynb', 'zapato_mike.png']
Loading Fashion MNIST dataset...


c:\Users\Miguel\Desktop\Miguelon\GH\Machine_Learning_Projects\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Training data shape: (60000, 28, 28, 1)
Test data shape: (10000, 28, 28, 1)


c:\Users\Miguel\Desktop\Miguelon\GH\Machine_Learning_Projects\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training model...
Epoch 1/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.6952 - loss: 0.8374 - val_accuracy: 0.8145 - val_loss: 0.4806
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.8044 - loss: 0.5361 - val_accuracy: 0.8547 - val_loss: 0.3906
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.8358 - loss: 0.4587 - val_accuracy: 0.8635 - val_loss: 0.3643
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.8534 - loss: 0.4166 - val_accuracy: 0.8733 - val_loss: 0.3353
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.8660 - loss: 0.3795 - val_accuracy: 0.8835 - val_loss: 0.3107

Test accuracy: 88.19%
Model saved as: mi_modelo.keras


In [2]:
# Load the trained model
print("Loading model...")
model_path = 'mi_modelo.keras'

if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path, compile=False)
    print(f"Model loaded successfully from: {model_path}")
else:
    print(f"Error: {model_path} not found. Run the training cell first!")
    model = None

def predecir_ropa(ruta_imagen):
    """Predict clothing type from an image."""
    if model is None:
        raise RuntimeError('Model is not loaded. Please run the training cell first.')
    
    # Preprocess image to match training: grayscale, invert, resize, normalize, batch
    img = Image.open(ruta_imagen).convert('L')
    img = ImageOps.invert(img)
    img = img.resize((28, 28))
    img_array = np.array(img) / 255.0
    img_array = img_array.reshape(1, 28, 28, 1)

    # Predict
    preds = model.predict(img_array, verbose=0)
    probs = np.asarray(preds)[0]

    clases = ['Camiseta', 'Pantalón', 'Jersey', 'Vestido', 'Abrigo',
              'Sandalia', 'Camisa', 'Zapatilla', 'Bolso', 'Bota']
    idx = int(np.argmax(probs))
    conf = float(np.max(probs))

    print('------------------------------------------------')
    print('Imagen:', ruta_imagen)
    print('Predicción:', clases[idx].upper())
    print('Confianza: {0:.2f}%'.format(conf * 100))
    print('Probabilities:', {clases[i]: f"{probs[i]*100:.1f}%" for i in range(10)})
    print('------------------------------------------------')
    return clases[idx], conf

Loading model...
Model loaded successfully from: mi_modelo.keras


In [4]:
# Test the model with the available image
print("Testing predictions...")
try:
    if os.path.exists('zapato_mike.png'):
        result, confidence = predecir_ropa('zapato_mike.png')
        print(f"\n✓ Successfully predicted: {result} (confidence: {confidence*100:.2f}%)")
    else:
        print("zapato_mike.png not found in current directory")
except Exception as e:
    print(f'Error al predecir: {e}')

Testing predictions...
------------------------------------------------
Imagen: zapato_mike.png
Predicción: BOLSO
Confianza: 96.45%
Probabilities: {'Camiseta': '1.0%', 'Pantalón': '0.5%', 'Jersey': '1.2%', 'Vestido': '0.2%', 'Abrigo': '0.1%', 'Sandalia': '0.3%', 'Camisa': '0.1%', 'Zapatilla': '0.0%', 'Bolso': '96.5%', 'Bota': '0.1%'}
------------------------------------------------

✓ Successfully predicted: Bolso (confidence: 96.45%)
